#RIC3 trainee project
###Code retreived from the Global Parkinson's Genetics Program (GP2)-Bioinformatics-course, Sara Bandres-Ciga, Hampton Leonard, Mary Makarious and other members of the International Parkinson's Disease Genomics Consortium (IPDGC)
###Modified by Kajsa Brolin, Manuela Tan

General info:
When setting up the cloud environment, use the "Legacy Python/R (default prior to January 14, 2020)". For unknown reason, the current default GATK 4.1 4.1... environment will not run properly. Errors occur when using plink (at least it did in January 2021) stating that the disk space is running out no matter how high you go (for me, this occured at the step of updating sex info despite setting disk size to 1 TB). Setting disk size to 500 GB has been enough for me. 

If this is your first time running a Terra Notebook, you should run either the Py3 - Start Here (or the R - Start Here you want to use R instead). I would also highly recommend to go trough the GP2 Introduction. Using Terra to Access Data and Perform Analyses : https://github.com/GP2-TNC-WG/GP2-Bioinformatics-course/blob/master/Introduction.md#4 

QC of the plink files has been performed (with minor modifications) according to the GP2 tutorial from: https://github.com/GP2-TNC-WG/GP2-Bioinformatics-course/blob/master/Module_I.md. Plotting the PCA results and creating the covariate file has however been done in R).

Once the QC is done, the rest is similar to the other IPDGC trainee scripts.

Please inform us if something doesn't work or if there is a better way of doing things! Not experts and want to learn :) 

###Getting Started
Run the cells below to set up libraries and functions.

We will also set up the billing project for making queries and set a few path variables to pull data.

In [1]:
#Set up libraries
# Use the os package to interact with the environment
import os

# Bring in Pandas for Dataframe functionality
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

# numpy for basics
import numpy as np

# Use StringIO for working with file contents
from io import StringIO

# Enable IPython to display matplotlib graphs
import matplotlib.pyplot as plt
%matplotlib inline

# Enable interaction with the FireCloud API
from firecloud import api as fapi

# Import the iPython HTML rendering for displaying links to Google Cloud Console
from IPython.core.display import display, HTML

# Import urllib modules for building URLs to Google Cloud Console
import urllib.parse

# BigQuery for querying data
from google.cloud import bigquery


###Set up billing project and data path variables

In [2]:
BILLING_PROJECT_ID = os.environ['GOOGLE_PROJECT']
WORKSPACE_NAMESPACE = os.environ['WORKSPACE_NAMESPACE']
WORKSPACE_NAME = os.environ['WORKSPACE_NAME']
WORKSPACE_BUCKET = os.environ['WORKSPACE_BUCKET']

WORKSPACE_ATTRIBUTES = fapi.get_workspace(WORKSPACE_NAMESPACE, WORKSPACE_NAME).json().get('workspace',{}).get('attributes',{})

GS_RELEASE_PATH = 'gs://amp-pd-data/releases/2019_v1release_1015'
GS_CLINICAL_RELEASE_PATH = f'{GS_RELEASE_PATH}/clinical'

GS_WGS_RELEASE_PATH = 'gs://amp-pd-genomics/releases/2019_v1release_1015/wgs'
GS_WGS_RELEASE_PLINK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'plink')
GS_WGS_RELEASE_GATK_PATH = os.path.join(GS_WGS_RELEASE_PATH, 'gatk')

BQ_RELEASE_DATASET = 'amp-pd-research.2019_v1release_1015'


print(BILLING_PROJECT_ID)
print(GS_CLINICAL_RELEASE_PATH)
print(GS_WGS_RELEASE_PLINK_PATH)
print(GS_WGS_RELEASE_GATK_PATH)

ipdgctraineeproject-ric3
gs://amp-pd-data/releases/2019_v1release_1015/clinical
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/gatk


In [3]:
##Some useful functions (which actually needs to be run in order for the rest to work)
#Utility routine for printing a shell command before executing it
def shell_do(command):
    print(f'Executing: {command}')
    !$command

# Utility routines for reading files from Google Cloud Storage
def gcs_read_file(path):
    """Return the contents of a file in GCS"""
    contents = !gsutil -u {BILLING_PROJECT_ID} cat {path}
    return '\n'.join(contents)
    
def gcs_read_csv(path, sep=None):
    """Return a DataFrame from the contents of a delimited file in GCS"""
    return pd.read_csv(StringIO(gcs_read_file(path)), sep=sep, engine='python')

# Utility routine for display a message and a link
def display_html_link(description, link_text, url):
    html = f'''
    <p>
    </p>
    <p>
    {description}
    <a target=_blank href="{url}">{link_text}</a>.
    </p>
    '''

    display(HTML(html))
    
# Utility routine for displaying a message and link to Cloud Console
def link_to_cloud_console_gcs(description, link_text, gcs_path):
    url = '{}?{}'.format(
        os.path.join('https://console.cloud.google.com/storage/browser',
                     gcs_path.replace("gs://","")),
        urllib.parse.urlencode({'userProject': BILLING_PROJECT_ID}))

    display_html_link(description, link_text, url)
    
# Get the data from a query
def bq_query(query):
    """Return the contents of a query against BigQuery"""
    return pd.read_gbq(
        query,
        project_id=BILLING_PROJECT_ID,
        dialect='standard')

###Querying Clinical Data
Let's look at all the available clinical tables in BigQuery. The following query will tell you the available dataset ID's, names, row count, and size.

In [4]:
clinical_tables = f"""

SELECT 
project_id, dataset_id, table_id, row_count, size_bytes 

FROM `{BQ_RELEASE_DATASET}.__TABLES__`

"""


bq_query(clinical_tables)

Downloading: 100%|██████████| 33/33 [00:00<00:00, 122.35rows/s]


,project_id,dataset_id,table_id,row_count,size_bytes
0,amp-pd-research,2019_v1release_1015,Biospecimen_analyses_CSF_abeta_tau_ptau,9385,458495
1,amp-pd-research,2019_v1release_1015,Biospecimen_analyses_CSF_beta_glucocerebrosidase,278,24342
2,amp-pd-research,2019_v1release_1015,Biospecimen_analyses_SomaLogic_plasma,26100,2273980
3,amp-pd-research,2019_v1release_1015,Biospecimen_analyses_other,6105,559618
4,amp-pd-research,2019_v1release_1015,Caffeine_history,1222,55586
5,amp-pd-research,2019_v1release_1015,DTI,1180,132014
6,amp-pd-research,2019_v1release_1015,DaTSCAN_SBR,1976,106071
7,amp-pd-research,2019_v1release_1015,DaTSCAN_visual_interpretation,842,27048
8,amp-pd-research,2019_v1release_1015,Demographics,4298,413191
9,amp-pd-research,2019_v1release_1015,Enrollment,4280,447363


In [5]:
demographics = f"""

SELECT 
participant_id, age_at_baseline, sex, race

FROM `{BQ_RELEASE_DATASET}.Demographics`

"""


demo = bq_query(demographics)

Downloading: 100%|██████████| 4298/4298 [00:00<00:00, 16025.27rows/s]


In [6]:
demo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 4 columns):
participant_id     4298 non-null object
age_at_baseline    4298 non-null int64
sex                4298 non-null object
race               4297 non-null object
dtypes: int64(1), object(3)
memory usage: 134.4+ KB


In [7]:
demo.head()

,participant_id,age_at_baseline,sex,race
0,PP-41564,22,Male,White
1,PD-PDZV843ATF,24,Male,White
2,PD-PDCK871NBR,24,Male,White
3,PP-51718,25,Male,White
4,PP-56267,25,Male,White


*Important: clinical duplicates*
In the clinical data there remain samples that have been determined to be genetically identical (either twins or simply sample duplication across studies).

The samples with lower coverage were removed from the WGS data. The two columns of the duplicated sample table below refer to the IDs of samples deleted from the WGS data (but still in the clinical data) and the IDs of samples that remain in the WGS data, respectively.

It is very important before running analyses using just the clinical data to investigate duplicates or you may be including the same sample twice. In addition, some of the duplicated samples from different studies have differing diagnoses.

In [8]:
dups = f"""

SELECT 
*

FROM `{BQ_RELEASE_DATASET}.duplicate_subjects`

"""


duplicates = bq_query(dups)

Downloading: 100%|██████████| 24/24 [00:00<00:00, 202.72rows/s]


In [9]:
duplicates

,deleted_participant_id,wgs_participant_id
0,BF-1091,PD-PDEB612LPE
1,BF-1098,PD-PDUA781LH0
2,BF-1125,PD-PDWY520TTB
3,HB-PD_INVFA733YEF,BF-1130
4,HB-PD_INVDZ260AW1,PD-PDFK551GW2
5,HB-PD_INVEP649ZE5,PD-PDGJ885TBH
6,HB-PD_INVME305HYH,PD-PDPV262JLC
7,HB-PD_INVWE905KPJ,PD-PDKF434UCX
8,HB-PD_INVNE231ZUD,PD-PDBR042ZHH
9,PD-PDBW494GHE,BF-1100


###Querying Variant Data

Information below is retreived from the GP2 introduction to work on Terra. We will not practice here but the infortmaion can be good to have read trough. 

"Now we will practice querying variant data. You should be more careful when querying this data, as it is much larger than the clinical datasets and might get expensive if you make multiple naive queries. In most cases, you should be querying from the gatk_passing_variants table, as this has been reformated to make queries less expensive. More info on the gatk_passing_variants is in the AMP-PD Getting Started notebooks:

The passing_variants table includes output from:

- The Broad Institute's joint genotyping workflow
- Variants annotated with the Variant Effect Predictor (VEP)
- The resulting VCFs from the above process were imported to BigQuery using the Variant Transforms pipeline. 

Further, the table was compacted and reshaped with:

- Variant calls matching the reference (genotype 0, 0) moved to a column (hom_ref_call)
- This column contbains only the sample identifiers, read statistics are omitted
- Variant calls which were no-calls (genotype -1, -1) moved to a column (no_call)
- This column contains only the sample identifiers read statistics are omitted
- Variants not matching FILTER = PASS have been omitted
Best practice is to target your variant queries. This means looking at the specific regions or variants you're interested in rather than querying the whole datatset."

###Pulling Data from Google Storage Buckets

Querying is a quick and easy way to pull the data you're interested in, but you can also read certain datasets directly from the Google Cloud Storage Buckets.
You can submit the command below to the shell using our path variables to look at the Google Bucket locations for the clinical data.

In [10]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_CLINICAL_RELEASE_PATH}')

Executing: gsutil -u ipdgctraineeproject-ric3 ls gs://amp-pd-data/releases/2019_v1release_1015/clinical
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Biospecimen_analyses_CSF_abeta_tau_ptau.csv
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Biospecimen_analyses_CSF_abeta_tau_ptau_dictionary.csv
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Biospecimen_analyses_CSF_beta_glucocerebrosidase.csv
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Biospecimen_analyses_CSF_beta_glucocerebrosidase_dictionary.csv
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Biospecimen_analyses_SomaLogic_plasma.csv
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Biospecimen_analyses_SomaLogic_plasma_dictionary.csv
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Biospecimen_analyses_other.csv
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Biospecimen_analyses_other_dictionary.csv
gs://amp-pd-data/releases/2019_v1release_1015/clinical/Caffeine_history.

Let's read in the Demographics CSV

In [11]:
demographics_df = gcs_read_csv(os.path.join(GS_CLINICAL_RELEASE_PATH, 'Demographics.csv'))

In [12]:
demographics_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4298 entries, 0 to 4297
Data columns (total 9 columns):
participant_id           4298 non-null object
GUID                     2688 non-null object
visit_name               4298 non-null object
visit_month              4298 non-null int64
age_at_baseline          4298 non-null int64
sex                      4298 non-null object
ethnicity                4297 non-null object
race                     4297 non-null object
education_level_years    4297 non-null object
dtypes: int64(2), object(7)
memory usage: 302.3+ KB


In [13]:
demographics_df.head()

,participant_id,GUID,visit_name,visit_month,age_at_baseline,sex,ethnicity,race,education_level_years
0,PD-PDNE299YPT,PDNE299YPT,M0,0,74,Male,Not Hispanic or Latino,White,Greater than 16 years
1,PD-PDYW828VAV,PDYW828VAV,M0,0,57,Female,Not Hispanic or Latino,White,12-16 years
2,PD-PDWV958JPG,PDWV958JPG,M0,0,68,Male,Not Hispanic or Latino,White,Greater than 16 years
3,PD-PDRF387ZV3,PDRF387ZV3,M0,0,68,Male,Not Hispanic or Latino,White,12-16 years
4,PD-PDAC268KWV,PDAC268KWV,M0,0,69,Female,Not Hispanic or Latino,White,Greater than 16 years


Some data formats aren't queryable, such as CRAMS, VCF files, and FASTQs

To use these data formats, we just need to copy the files to our cluster
First let's look at the locations in the cloud for these files:

In [14]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_WGS_RELEASE_PATH}')

Executing: gsutil -u ipdgctraineeproject-ric3 ls gs://amp-pd-genomics/releases/2019_v1release_1015/wgs
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/wgs_samples.csv
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/gatk/
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink2/
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/topmed/


In [16]:
shell_do(f'gsutil -u {BILLING_PROJECT_ID} ls {GS_WGS_RELEASE_PLINK_PATH}/bfile/all_vcfs*')

Executing: gsutil -u ipdgctraineeproject-ric3 ls gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs*
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bed
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bim
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.fam
gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.log


Now we can practice copying data from the cloud storage to our cluster. We'll need the Plink binary files for our analysis in a bit, so let's copy those.

First let's make a directory to copy it to.

In [17]:
%%bash

mkdir ~/bin
mkdir ~/bin/data_temp
cd ~/bin/
ls

data_temp


###Copying the vcf plink files:

In [18]:
shell_do(f'gsutil -mu {BILLING_PROJECT_ID} cp {GS_WGS_RELEASE_PLINK_PATH}/bfile/all_vcfs* ~/bin/data_temp')

Executing: gsutil -mu ipdgctraineeproject-ric3 cp gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs* ~/bin/data_temp
Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bed...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.bim...
Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.fam...
Copying gs://amp-pd-genomics/releases/2019_v1release_1015/wgs/plink/bfile/all_vcfs.log...
\ [4/4 files][ 21.4 GiB/ 21.4 GiB] 100% Done     0.0 B/s                        
Operation completed over 4 objects/21.4 GiB.                                     


In [20]:
%%bash
cd ~/bin/data_temp
ls

all_vcfs.bed
all_vcfs.bim
all_vcfs.fam
all_vcfs.log


In [21]:
%%bash
cd ~/bin/data_temp
head all_vcfs.fam

BF-1001	BF-1001	0	0	0	-9
BF-1002	BF-1002	0	0	0	-9
BF-1003	BF-1003	0	0	0	-9
BF-1004	BF-1004	0	0	0	-9
BF-1005	BF-1005	0	0	0	-9
BF-1006	BF-1006	0	0	0	-9
BF-1008	BF-1008	0	0	0	-9
BF-1009	BF-1009	0	0	0	-9
BF-1010	BF-1010	0	0	0	-9
BF-1011	BF-1011	0	0	0	-9


--> We need to update the fam file

Using the Plink binary files that we copied to our cluster previously, we'll do a Plink case/control association test. For this, we will be using the fisher flag, which performs Fisher's exact test. Fisher's test is used to determine if there are nonrandom associations between two variables. Our variables here are the presence of a Parkinon's Disease diagnosis (1 if yes, 0 if no), and the major and minor allele distribution for the tested variant. So for every variant in our data, we are testing whether the distribution of alleles is significantly different between the PD group and the non PD group.

To do this you'll need a few extra things:

Plink installed on your cluster
Phenotype file
Sex info file
Covariate file
Installing your own packages
You can get package files ready for installation in different ways.
Get the files from online, upload them to the workspace bucket, copy them to the cluster.
Download straight from a code cell using commands like git clone.
You will then follow the regular instructions for installation.
We are going to use wget to grab Plink 1.9 for Linux from http://s3.amazonaws.com

In [19]:
%%bash

if test -e ~/bin/plink; then

echo "Plink is already installed in ~/bin"
else
echo "Plink is not installed"
cd ~/bin/

wget http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip 

unzip -o plink_linux_x86_64_20190304.zip

fi

Plink is not installed
Archive:  plink_linux_x86_64_20190304.zip
  inflating: plink                   
  inflating: LICENSE                 
  inflating: toy.ped                 
  inflating: toy.map                 
  inflating: prettify                


--2021-02-02 08:19:38--  http://s3.amazonaws.com/plink1-assets/plink_linux_x86_64_20190304.zip
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.107.182
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.107.182|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8708135 (8.3M) [application/zip]
Saving to: ‘plink_linux_x86_64_20190304.zip’

     0K .......... .......... .......... .......... ..........  0%  839K 10s
    50K .......... .......... .......... .......... ..........  1% 1.64M 8s
   100K .......... .......... .......... .......... ..........  1%  160M 5s
   150K .......... .......... .......... .......... ..........  2% 26.7M 4s
   200K .......... .......... .......... .......... ..........  2% 1.73M 4s
   250K .......... .......... .......... .......... ..........  3% 79.5M 3s
   300K .......... .......... .......... .......... ..........  4% 57.2M 3s
   350K .......... .......... .......... .......... ..........  4% 38.8M 2s
   400K .......

In [22]:
!~/bin/plink

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3

  plink <input flag(s)...> [command flag(s)...] [other flag(s)...]
  plink --help [flag name(s)...]

Commands include --make-bed, --recode, --flip-scan, --merge-list,
--write-snplist, --list-duplicate-vars, --freqx, --missing, --test-mishap,
--hardy, --mendel, --ibc, --impute-sex, --indep-pairphase, --r2, --show-tags,
--blocks, --distance, --genome, --homozyg, --make-rel, --make-grm-gz,
--rel-cutoff, --cluster, --pca, --neighbour, --ibs-test, --regress-distance,
--model, --bd, --gxe, --logistic, --dosage, --lasso, --test-missing,
--make-perm-pheno, --tdt, --qfam, --annotate, --clump, --gene-report,
--meta-analysis, --epistasis, --fast-epistasis, and --score.

'plink --help | more' describes all functions (warning: long).


Now we need our phenotype and covariate files. I'm going to just query a few data fields like we did before. I'm just going to grab diagnosis and sex for this quick example.

Tip: Get to know the cohorts before running analyses, cohorts like PPMI have study arms that are enriched for certain variants and you may need to exclude those samples

We need to grab sex information to update the Plink fam file, as it currently does not have sex coded into it. Chi squared tests or Fisher's tests like Plink's association tests don't use covariates, but Plink still needs sex info or it sets those samples to missing. We'll grab the sex info from the demographics file and use it to update the Plink fam file.

Will also grab phenotype information and update the plink fam file

In [23]:
covariates = f"""

SELECT 
participant_id, sex, age_at_baseline 

FROM `{BQ_RELEASE_DATASET}.Demographics`

"""


covs = bq_query(covariates)

Downloading: 100%|██████████| 4298/4298 [00:00<00:00, 23411.36rows/s]


In [24]:
covs.head()

,participant_id,sex,age_at_baseline
0,PP-41564,Male,22
1,PD-PDZV843ATF,Male,24
2,PD-PDCK871NBR,Male,24
3,PP-51718,Male,25
4,PP-56267,Male,25


There is a case control table available that includes baseline and most recent diagnosis. Healthy samples are controls, any Parkinson's disease or idiopathic Parkinson's diagnsoses are classified as cases, and any other related diagnoses are classified as other.

In [25]:
phenotype = f"""

SELECT 
* 

FROM `{BQ_RELEASE_DATASET}.amp_pd_case_control`

"""

pheno = bq_query(phenotype)

Downloading: 100%|██████████| 4298/4298 [00:00<00:00, 14032.50rows/s]


In [26]:
pheno.head()

,participant_id,diagnosis_at_baseline,diagnosis_latest,case_control_other_at_baseline,case_control_other_latest
0,PD-PDAB597LKL,Parkinsonism,Parkinsonism,Other,Other
1,PD-PDCT406CN9,Parkinsonism,Parkinsonism,Other,Other
2,PP-54461,Prodromal motor PD,Idiopathic PD,Other,Case
3,PP-54215,Prodromal motor PD,Idiopathic PD,Other,Case
4,PP-18567,Prodromal motor PD,Idiopathic PD,Other,Case


To use Plink, we need to get our sex info file and phenotype file in Plink format, which means 2 columns of IDs followed by the other information.

In [27]:
covsp = covs
covsp['IID'] = covsp['participant_id']
covsp = covsp[['participant_id', 'IID', 'sex', 'age_at_baseline']]
covsp['sex'] = covsp['sex'].astype(str)
covsp.sex[(covsp.sex == "Male")] = 1
covsp.sex[(covsp.sex == "Female")] = 2
covsp.columns = ['FID','IID', 'sex', 'age_at_baseline']

In [28]:
covsp.head()

,FID,IID,sex,age_at_baseline
0,PP-41564,PP-41564,1,22
1,PD-PDZV843ATF,PD-PDZV843ATF,1,24
2,PD-PDCK871NBR,PD-PDCK871NBR,1,24
3,PP-51718,PP-51718,1,25
4,PP-56267,PP-56267,1,25


In [29]:
phenop = pheno
phenop['IID'] = phenop['participant_id']
phenop = phenop[['participant_id', 'IID', 'case_control_other_latest']]
phenop.columns = ['FID','IID', 'PHENO']
phenop = phenop[(phenop.PHENO != 'Other')]
phenop.PHENO[(phenop.PHENO == "Case")] = 2
phenop.PHENO[(phenop.PHENO == "Control")] = 1

In [30]:
phenop.head()

,FID,IID,PHENO
2,PP-54461,PP-54461,2
3,PP-54215,PP-54215,2
4,PP-18567,PP-18567,2
5,PP-40615,PP-40615,2
6,PP-40612,PP-40612,2


Let's write these to tab files so we can use them with Plink

In [31]:
with open('plink_test_covs.tab', 'w') as f:
    f.write(covsp.to_csv(index=False, sep='\t'))

In [32]:
with open('plink_test_pheno.tab', 'w') as f:
    f.write(phenop.to_csv(index=False, sep='\t'))

Now we'll use Plink's --update-sex command to update the fam file with the information we queried from the demographics file.(It will be a mix of running plink in python vs bash below, sorry abut that)

In [38]:
shell_do(f'~/bin/plink --bfile ~/bin/data_temp/all_vcfs \
            --update-sex /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit/plink_test_covs.tab \
            --make-bed \
            --out ~/bin/data_temp/plink_updated_sex \
            --threads 2')

Executing: ~/bin/plink --bfile ~/bin/data_temp/all_vcfs             --update-sex /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit/plink_test_covs.tab             --make-bed             --out ~/bin/data_temp/plink_updated_sex             --threads 2
PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to /home/jupyter-user/bin/data_temp/plink_updated_sex.log.
Options in effect:
  --bfile /home/jupyter-user/bin/data_temp/all_vcfs
  --make-bed
  --out /home/jupyter-user/bin/data_temp/plink_updated_sex
  --threads 2
  --update-sex /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit/plink_test_covs.tab

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
3074 people (0 males, 0 females, 3074 ambiguous) loaded from .fam.
Ambiguous sex IDs written to
/home/jupyter-user/bin/data_temp/plink_updated_sex.nosex .
--upda

###Update phenotypes and remove individuals with missing phenotypes. 
It might be that this is not needed if you use the --pheno flag and a file with information on phenotypes for your analyses but I choose to add it here

Information from plink website:
https://www.cog-genomics.org/plink/1.9/input#pheno_encoding
Case/control phenotypes are expected to be encoded as 1=unaffected (control), 2=affected (case); 0 is accepted as an alternate missing value encoding. If you use the --1 flag, 0 is interpreted as unaffected status instead, while 1 maps to affected. This also forces phenotypes to be interpreted as case/control.

Case/control phenotype generation
--make-pheno <filename> <value>

Given a text file listing family and individual IDs in the first two columns, "--make-pheno [filename] '*'" designates all samples listed in the named file as cases, and all other samples as controls.

If the named file has a third column, and a value other than '*' is given, --make-pheno will designate all samples with third column entry equal to the given value as cases, all other samples mentioned in the file as controls, and all samples missing from the file as having missing phenotypes.

Use the plink_test_pheno.tab for phenotype info and remove individuals with missing phenotypes

In [39]:
!~/bin/plink --bfile ~/bin/data_temp/plink_updated_sex --make-pheno plink_test_pheno.tab 2 --keep plink_test_pheno.tab --make-bed --out AMPPD_updated_pheno

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_updated_pheno.log.
Options in effect:
  --bfile /home/jupyter-user/bin/data_temp/plink_updated_sex
  --keep plink_test_pheno.tab
  --make-bed
  --make-pheno plink_test_pheno.tab 2
  --out AMPPD_updated_pheno

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
3074 people (1721 males, 1353 females) loaded from .fam.
--make-pheno: 2835 phenotype values set.
--keep: 2835 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2835 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
many commands treat these as missing.


From GP2 Introduction to Terra course:
"Best practice would be to perform variant QC beforehand, such as removing missing by case/control and filtering controls by Hardy-Weinberg equilibrium. No variant QC has been done to these plink files other than removing variants that failed GATK pipeline QC, so that specific choices could be left up to individual researchers."

We decided to run a GWAS QC according to GP2s instructions

##Sample QC - Genotyping call rates

###Calculate genotyping call rates per sample

In [40]:
!~/bin/plink --bfile AMPPD_updated_pheno --missing --out call_rates

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to call_rates.log.
Options in effect:
  --bfile AMPPD_updated_pheno
  --missing
  --out call_rates

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2835 people (1577 males, 1258 females) loaded from .fam.
2835 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2835 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.993967.
--missing: Sample missing data report written to call_rates.imiss, and
variant-based missing dat

###Remove call rate outliers

In [41]:
!~/bin/plink --bfile AMPPD_updated_pheno --mind 0.05 --make-bed --out AMPPD_call_rate

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_call_rate.log.
Options in effect:
  --bfile AMPPD_updated_pheno
  --make-bed
  --mind 0.05
  --out AMPPD_call_rate

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2835 people (1577 males, 1258 females) loaded from .fam.
2835 phenotype values loaded from .fam.
0 people removed due to missing genotype data (--mind).
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2835 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
many commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.993967.
28791969 v

NOTES:

All call rates outliers are in CALL_RATE_OUTLIERS.txt
call_rates.imiss and F_MISS - 1 = callrate
All call rates saved in CALL_RATES_ALL_SAMPLES.txt

###Sample QC - Heterozygosity

In [42]:
!~/bin/plink --bfile AMPPD_updated_pheno --geno 0.01 --maf 0.05 --indep-pairwise 50 5 0.5 --out pruning
!~/bin/plink --bfile AMPPD_updated_pheno --extract pruning.prune.in --make-bed --out pruned_data
!~/bin/plink --bfile pruned_data --het --out prunedHet

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pruning.log.
Options in effect:
  --bfile AMPPD_updated_pheno
  --geno 0.01
  --indep-pairwise 50 5 0.5
  --maf 0.05
  --out pruning

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2835 people (1577 males, 1258 females) loaded from .fam.
2835 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2835 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.993967.
907891 variants removed due to missing genotype data (--gen

In [43]:
%%bash
awk '{if ($6 <= -0.15) print $0 }' prunedHet.het > outliers1.txt
awk '{if ($6 >= 0.15) print $0 }' prunedHet.het > outliers2.txt
cat outliers1.txt outliers2.txt > HETEROZYGOSITY_OUTLIERS.txt

cut -f 1,2 HETEROZYGOSITY_OUTLIERS.txt > all_outliers.txt

mv prunedHet.het HETEROZYGOSITY_DATA.txt

In [44]:
!~/bin/plink --bfile AMPPD_call_rate --remove all_outliers.txt --make-bed --out AMPPD_after_heterozyg

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_after_heterozyg.log.
Options in effect:
  --bfile AMPPD_call_rate
  --make-bed
  --out AMPPD_after_heterozyg
  --remove all_outliers.txt

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2835 people (1577 males, 1258 females) loaded from .fam.
2835 phenotype values loaded from .fam.
--remove: 2835 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2835 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
); many commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.993967.
28791969

NOTES:

--het from LD pruned data > use F cut-off of -0.15 and <- 0.15 for inclusion
outliers stored here -> all_outliers.txt
all heterozygosity is stored here -> HETEROZYGOSITY_DATA.txt

###Sample QC - Gender checking

In [45]:
!~/bin/plink --bfile AMPPD_after_heterozyg --check-sex 0.25 0.75 --maf 0.05 --out gender_check1
!~/bin/plink --bfile AMPPD_after_heterozyg --chr 23 --from-bp 2781479 --to-bp 155701382 --maf 0.05 --geno 0.05 --hwe 1E-5 --check-sex  0.25 0.75 --out gender_check2 

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to gender_check1.log.
Options in effect:
  --bfile AMPPD_after_heterozyg
  --check-sex 0.25 0.75
  --maf 0.05
  --out gender_check1

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2835 people (1577 males, 1258 females) loaded from .fam.
2835 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2835 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.993967.
21346980 variants removed due to minor allele threshold(s)
(--m

NOTES:

Gender failures are stored in GENDER_FAILURES.txt
Gender checks are stored in GENDER_CHECK1.txt and GENDER_CHECK2.txt

In [46]:
%%bash
grep "PROBLEM" gender_check1.sexcheck > problems1.txt
grep "PROBLEM" gender_check2.sexcheck > problems2.txt
cat problems1.txt problems2.txt > GENDER_FAILURES.txt
cut -f 1,2 GENDER_FAILURES.txt > samples_to_remove.txt

In [47]:
!~/bin/plink --bfile AMPPD_after_heterozyg --remove samples_to_remove.txt --make-bed --out AMPPD_after_gender

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_after_gender.log.
Options in effect:
  --bfile AMPPD_after_heterozyg
  --make-bed
  --out AMPPD_after_gender
  --remove samples_to_remove.txt

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2835 people (1577 males, 1258 females) loaded from .fam.
2835 phenotype values loaded from .fam.
--remove: 2834 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2834 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
many commands treat these as missing.
treat these as missing.
Total genotyping rate in remaining sample

###Sample QC - Ancestry

NOTES
No ancestry outliers -> based on Hapmap3 PCA plot, should be near combined CEU/TSI
Keep in mind that this comparison with hapmap is based on the number of SNPs that overlap between your input dataset and hapmap

NOTE! You need to upload the Hapmap files manually to your data folder (make sure that the gemone build is the same!)

In [48]:
%%bash
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/HAPMAP_GRCh38.fam /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/HAPMAP_GRCh38.bim /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/HAPMAP_GRCh38.bed /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit

Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/HAPMAP_GRCh38.fam...
/ [1 files][ 21.4 KiB/ 21.4 KiB]                                                
Operation completed over 1 objects/21.4 KiB.                                     
Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/HAPMAP_GRCh38.bim...
/ [1 files][ 29.3 MiB/ 29.3 MiB]                                                
Operation completed over 1 objects/29.3 MiB.                                     
Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/HAPMAP_GRCh38.bed...
==> NOTE: You are downloading one or more large file(s), which would
run significantly faster if you enabled sliced object downloads. This
feature is enabled by default but requires that compiled crcmod be
installed (see "gsutil help crcmod").

\ [1 files][168.0 MiB/168.0 MiB]                                                
Operation completed over 1 objects/168.0 MiB.                                    


In [49]:
ls 

=1.13.0                    HAPMAP_GRCh38.bim
all_outliers.txt           HAPMAP_GRCh38.fam
AMPPD_after_gender.bed     HETEROZYGOSITY_DATA.txt
AMPPD_after_gender.bim     HETEROZYGOSITY_OUTLIERS.txt
AMPPD_after_gender.fam     outliers1.txt
AMPPD_after_gender.hh      outliers2.txt
AMPPD_after_gender.log     plink_test_covs.tab
AMPPD_after_heterozyg.bed  plink_test_pheno.tab
AMPPD_after_heterozyg.bim  problems1.txt
AMPPD_after_heterozyg.fam  problems2.txt
AMPPD_after_heterozyg.hh   pruned_data.bed
AMPPD_after_heterozyg.log  pruned_data.bim
AMPPD_call_rate.bed        pruned_data.fam
AMPPD_call_rate.bim        pruned_data.hh
AMPPD_call_rate.fam        pruned_data.log
AMPPD_call_rate.hh         prunedHet.hh
AMPPD_call_rate.log        prunedHet.log
AMPPD_updated_pheno.bed    pruning.hh
AMPPD_updated_pheno.bim    pruning.log
AMPPD_updated_pheno.fam    pruning.prune.in
AMPPD_updated_pheno.hh     pruning.prune.out
AMPPD_updated_pheno.log    Py3 - Clinical - load a CSV from Cloud Storage.ipynb
call

In [50]:
!~/bin/plink --bfile AMPPD_after_gender --bmerge HAPMAP_GRCh38 --out hapmap3_bin_snplis --make-bed
!~/bin/plink --bfile AMPPD_after_gender --flip hapmap3_bin_snplis-merge.missnp --make-bed --out after_gender3
!~/bin/plink --bfile after_gender3 --bmerge HAPMAP_GRCh38 --out hapmap3_bin_snplis --make-bed
!~/bin/plink --bfile after_gender3 --exclude hapmap3_bin_snplis-merge.missnp --out after_gender4 --make-bed
!~/bin/plink --bfile after_gender4 --bmerge HAPMAP_GRCh38 --out hapmap3_bin_snplis --make-bed
!~/bin/plink --bfile hapmap3_bin_snplis --geno 0.01 --maf 0.05 --indep-pairwise 50 5 0.5 --out pruning
!~/bin/plink --bfile hapmap3_bin_snplis --extract pruning.prune.in --make-bed --out pruned_data
!~/bin/plink --bfile pruned_data --het --out prunedHet 

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to hapmap3_bin_snplis.log.
Options in effect:
  --bfile AMPPD_after_gender
  --bmerge HAPMAP_GRCh38
  --make-bed
  --out hapmap3_bin_snplis

60402 MB RAM detected; reserving 30201 MB for main workspace.
2834 people loaded from AMPPD_after_gender.fam.
640 people to be merged from HAPMAP_GRCh38.fam.
Of these, 640 are new, while 0 are present in the base dataset.
28791969 markers loaded from AMPPD_after_gender.bim.
1100686 markers to be merged from HAPMAP_GRCh38.bim.
Of these, 9352 are new, while 1091334 are present in the base dataset.
82 more multiple-position warnings: see log file.
Error: 1647 variants with 3+ alleles present.
* If you believe this is due to strand inconsistency, try --flip with
  hapmap3_bin_snplis-merge.missnp.
  (Warning: if this seems to work, strand errors involving SNPs with A/T or C/G
  alleles pro

28801321 variants loaded from .bim file.
3474 people (1895 males, 1579 females) loaded from .fam.
2834 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 3474 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
many commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.817621.
28801321 variants and 3474 people pass filters and QC.
Among remaining phenotypes, 1734 are cases and 1100 are controls.  (640
phenotypes are missing.)
--make-bed to hapmap3_bin_snplis.bed + hapmap3_bin_snplis.bim +
hapmap3_bin_snplis.fam ... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697

In [51]:
!~/bin/plink --bfile pruned_data --geno 0.01 --out pca --make-bed --pca

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pca.log.
Options in effect:
  --bfile pruned_data
  --geno 0.01
  --make-bed
  --out pca
  --pca

60402 MB RAM detected; reserving 30201 MB for main workspace.
277894 variants loaded from .bim file.
3474 people (1895 males, 1579 females) loaded from .fam.
2834 phenotype values loaded from .fam.
Using up to 15 threads (change this with --threads).
Before main variant filters, 3474 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999717.
0 variants removed due to missing genotype data (--geno).
277894 variants and 3474 people pass filters and QC.
Among remaining phenotypes, 1734 are cases and

Files needed for the next step needs to be manually uploaded to data. These were downloaded from: https://github.com/neurogenetics/GWAS-pipeline/blob/master/ancestry_comparison

In [52]:
%%bash
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/eur_add.txt /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/asia_add.txt /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/afri_add.txt /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/PCA_in_R.R /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit

Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/eur_add.txt...
/ [1 files][  389.0 B/  389.0 B]                                                
Operation completed over 1 objects/389.0 B.                                      
Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/asia_add.txt...
/ [1 files][  493.0 B/  493.0 B]                                                
Operation completed over 1 objects/493.0 B.                                      
Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/afri_add.txt...
/ [1 files][  395.0 B/  395.0 B]                                                
Operation completed over 1 objects/395.0 B.                                      
Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/PCA_in_R.R...
/ [1 files][  3.3 KiB/  3.3 KiB]                                                
Operation completed over 1 objects/3.3 KiB.                                      


In [53]:
%%bash
grep "EUROPE" pca.eigenvec > eur.txt
grep "ASIA" pca.eigenvec > asia.txt
grep "AFRICA" pca.eigenvec > afri.txt
grep -v -f eur.txt pca.eigenvec | grep -v -f asia.txt | grep -v -f afri.txt > new_samples.txt
cut -d " " -f 3 AMPPD_after_gender.fam > new_samples_add.txt
paste new_samples_add.txt new_samples.txt > new_samples2.txt
paste eur_add.txt eur.txt > euro.txt
paste asia_add.txt asia.txt > asiao.txt
paste afri_add.txt afri.txt > afrio.txt
cat new_samples2.txt euro.txt asiao.txt afrio.txt > pca.eigenvec2

I have used R to plot the PCA and filter the individuals according to the code below. Have not set it up to run directly in this notebook though. 

%%R
##1) PCA

#Running PCA script from: https://github.com/neurogenetics/GWAS-pipeline/tree/master/ancestry_comparison 

all_data <- read.table("pca.eigenvec2", header = F, fill=T)
all_data$V1[all_data$V1 == 0] <- 4
all_data$V1[all_data$V1 == 1] <- 5

attach(all_data)
all_data <- all_data[order(V1),] 

pdf("raw_hapmap_plot.pdf")
plot(all_data$V4,all_data$V5,pch=16,col=all_data$V1,xlab="PC1",ylab="PC2")
legend("bottomleft",col=c("green3","red","blue","cyan"),pch=16,c("Africa","Asia","New samples","Europe"))
dev.off()

project <- Sys.getenv('WORKSPACE_NAMESPACE')
workspace <- Sys.getenv('WORKSPACE_NAME')
bucket <- Sys.getenv('WORKSPACE_BUCKET')

euros <- read.table("euro.txt", header = F)
asians <- read.table("asiao.txt", header = F)
africans <- read.table("afrio.txt", header = F)

eurosLowC1 <- mean(euros$V4) - (6*sd(euros$V4))
eurosHighC1 <- mean(euros$V4) + (6*sd(euros$V4))
eurosLowC2 <- mean(euros$V5) - (6*sd(euros$V5))
eurosHighC2 <- mean(euros$V5) + (6*sd(euros$V5))

asiansLowC1 <- mean(asians$V4) - (6*sd(asians$V4))
asiansHighC1 <- mean(asians$V4) + (6*sd(asians$V4))
asiansLowC2 <- mean(asians$V5) - (6*sd(asians$V5))
asiansHighC2 <- mean(asians$V5) + (6*sd(asians$V5))

africansLowC1 <- mean(africans$V4) - (6*sd(africans$V4))
africansHighC1 <- mean(africans$V4) + (6*sd(africans$V4))
africansLowC2 <- mean(africans$V5) - (6*sd(africans$V5))
africansHighC2 <- mean(africans$V5) + (6*sd(africans$V5))

temp2 = all_data[(all_data$V4>=eurosLowC1),]
temp3 = temp2[(temp2$V4<=eurosHighC1),]
temp4 = temp3[(temp3$V5>=eurosLowC2),]
EURO = temp4[(temp4$V5<=eurosHighC2),]

pdf("EURO_confirmation_plot.pdf")
plot(EURO$V4,EURO$V5,pch=16,col=EURO$V1)
legend("topright",col=c("blue","cyan"),pch=16,c("New samples","Europe"))
dev.off()

temp2 = all_data[(all_data$V4>=asiansLowC1),]
temp3 = temp2[(temp2$V4<=asiansHighC1),]
temp4 = temp3[(temp3$V5>=asiansLowC2),]
ASIA = temp4[(temp4$V5<=asiansHighC2),]

pdf("ASIA_confirmation_plot.pdf")
plot(ASIA$V4,ASIA$V5,pch=16,col=ASIA$V1)
legend("topright",col=c("blue","red"),pch=16,c("New samples","Asia"))
dev.off()

temp2 = all_data[(all_data$V4>=africansLowC1),]
temp3 = temp2[(temp2$V4<=africansHighC1),]
temp4 = temp3[(temp3$V5>=africansLowC2),]
AFRICA = temp4[(temp4$V5<=africansHighC2),]

pdf("AFRICA_confirmation_plot.pdf")
plot(AFRICA$V4,AFRICA$V5,pch=16,col=AFRICA$V1)
legend("topleft",col=c("blue","green3"),pch=16,c("New samples","Africa"))
dev.off()

library(dplyr)

main_data2 <- all_data[ ! all_data %in% EURO ]

mixed_race1 <- anti_join(all_data, EURO)
mixed_race2 <- anti_join(mixed_race1, ASIA)
mixed_race3 <- anti_join(mixed_race2, AFRICA)

total <- rbind(euros,asians,africans,mixed_race3)

pdf("mixed_race_confirmation_plot.pdf")
plot(total$V4,total$V5,pch=16,col=total$V1)
legend("topleft",col=c("green3","red","blue","black"),pch=16,c("Africa","Asia","Mixed race samples","Europe"))
dev.off()

final_euro = subset(EURO, EURO$V1 != 5)
final_euro2 =  data.frame(final_euro$V2,final_euro$V3)

final_asia = subset(ASIA, ASIA$V1 != 2)
final_asia2 =  data.frame(final_asia$V2,final_asia$V3)

final_africa = subset(AFRICA, AFRICA$V1 != 3)
final_africa2 =  data.frame(final_africa$V2,final_africa$V3)

mixed_race4 =  data.frame(mixed_race3$V2,mixed_race3$V3)

write.table(final_euro2,file = "PCA_filtered_europeans.txt", quote = FALSE,row.names=F,col.names = F)
write.table(final_asia2,file = "PCA_filtered_asians.txt", quote = FALSE,row.names=F,col.names = F)
write.table(final_africa2,file = "PCA_filtered_africans.txt", quote = FALSE,row.names=F,col.names = F)
write.table(mixed_race4,file = "PCA_filtered_mixed_race.txt", quote = FALSE,row.names=F,col.names = F)

In [1]:
ls

=1.13.0
afri_add.txt
AFRICA_confirmation_plot.pdf
afrio.txt
afri.txt
after_gender3.bed
after_gender3.bim
after_gender3.fam
after_gender3.hh
after_gender3.log
after_gender4.bed
after_gender4.bim
after_gender4.fam
after_gender4.hh
after_gender4.log
all_outliers.txt
AMPPD_after_gender.bed
AMPPD_after_gender.bim
AMPPD_after_gender.fam
AMPPD_after_gender.hh
AMPPD_after_gender.log
AMPPD_after_heterozyg.bed
AMPPD_after_heterozyg.bim
AMPPD_after_heterozyg.fam
AMPPD_after_heterozyg.hh
AMPPD_after_heterozyg.log
AMPPD_call_rate.bed
AMPPD_call_rate.bim
AMPPD_call_rate.fam
AMPPD_call_rate.hh
AMPPD_call_rate.log
AMPPD_updated_pheno.bed
AMPPD_updated_pheno.bim
AMPPD_updated_pheno.fam
AMPPD_updated_pheno.hh
AMPPD_updated_pheno.log
asia_add.txt
ASIA_confirmation_plot.pdf
asiao.txt
asia.txt
call_rates.hh
call_rates.imiss
call_rates.lmiss
call_rates.log
eur_add.txt
EURO_confirmation_plot.pdf
euro.txt
eur.txt
facets/
gender_check1.hh
gender_check1.log
gender_check1.sexcheck
gender_check2.hh
gender_check2.

NOTES:

Extract population of interest. In this case europeans, but can be any populations that is present in base comparison dataset
This creates several plots and lists based on genetic ancestry

In [2]:
!~/bin/plink --bfile AMPPD_after_gender --keep PCA_filtered_europeans.txt --make-bed --out AMPPD_after_gender_heterozyg_hapmap

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_after_gender_heterozyg_hapmap.log.
Options in effect:
  --bfile AMPPD_after_gender
  --keep PCA_filtered_europeans.txt
  --make-bed
  --out AMPPD_after_gender_heterozyg_hapmap

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2834 people (1577 males, 1257 females) loaded from .fam.
2834 phenotype values loaded from .fam.
--keep: 2688 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2688 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
AMPPD_after_gender_heterozyg_hapmap.hh ); many commands treat these as 

In [3]:
%%bash
cat PCA_filtered_asians.txt PCA_filtered_africans.txt PCA_filtered_mixed_race.txt > hapmap_outliers33.txt

Download the plots from the PCA to you data

In [4]:
!gsutil cp ./raw_hapmap_plot.pdf gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./EURO_confirmation_plot.pdf gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./ASIA_confirmation_plot.pdf gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AFRICA_confirmation_plot.pdf gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./mixed_race_confirmation_plot.pdf gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf

Copying file://./raw_hapmap_plot.pdf [Content-Type=application/pdf]...
/ [1 files][ 24.5 KiB/ 24.5 KiB]                                                
Operation completed over 1 objects/24.5 KiB.                                     
Copying file://./EURO_confirmation_plot.pdf [Content-Type=application/pdf]...
/ [1 files][ 24.3 KiB/ 24.3 KiB]                                                
Operation completed over 1 objects/24.3 KiB.                                     
Copying file://./ASIA_confirmation_plot.pdf [Content-Type=application/pdf]...
/ [1 files][  6.6 KiB/  6.6 KiB]                                                
Operation completed over 1 objects/6.6 KiB.                                      
Copying file://./AFRICA_confirmation_plot.pdf [Content-Type=application/pdf]...
/ [1 files][  6.1 KiB/  6.1 KiB]                                                
Operation completed over 1 objects/6.1 KiB.                                      
Copying file://./mixed_race_confirmation_

To check that the files have been downloaded:

In [6]:
!gsutil ls gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf

gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/=1.13.0
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AFRICA_confirmation_plot.pdf
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB.CMC.assoc
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB.CMCWald.assoc
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB.Fp.assoc
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB.MadsonBrowning.assoc
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB.Skat.assoc
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB.SkatO.assoc
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB.Zeggini.assoc
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB.log
gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/AMP_PD_BURDEN.RIC3.WGS.maf003_KB

###Sample QC - Relatedness

NOTES:

PIHAT threshold of 0.125
GCTA needs to be installed!!

In [7]:
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap --geno 0.01 --maf 0.05 --indep-pairwise 50 5 0.5 --out pruning
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap --extract pruning.prune.in --make-bed --out pruned_data
!~/bin/plink --bfile pruned_data --het --out prunedHet

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to pruning.log.
Options in effect:
  --bfile AMPPD_after_gender_heterozyg_hapmap
  --geno 0.01
  --indep-pairwise 50 5 0.5
  --maf 0.05
  --out pruning

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2688 people (1503 males, 1185 females) loaded from .fam.
2688 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2688 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.99401.
891874 variants removed due to missing genot

In [8]:
%%bash

if test -e ~/bin/gcta64; then

echo "GCTA is already installed in ~/bin"
else
echo "GCTA is not installed"
cd ~/bin/

wget https://cnsgenomics.com/software/gcta/bin/gcta_1.93.2beta.zip

unzip -o gcta_1.93.2beta.zip

fi

GCTA is not installed
Archive:  gcta_1.93.2beta.zip
   creating: gcta_1.93.2beta/
  inflating: gcta_1.93.2beta/gcta64  
  inflating: gcta_1.93.2beta/.DS_Store  
  inflating: gcta_1.93.2beta/test.bim  
  inflating: gcta_1.93.2beta/MIT_License.txt  
  inflating: gcta_1.93.2beta/test.fam  
  inflating: gcta_1.93.2beta/README.txt  
  inflating: gcta_1.93.2beta/test.bed  
  inflating: gcta_1.93.2beta/test.phen  


--2021-02-02 12:49:24--  https://cnsgenomics.com/software/gcta/bin/gcta_1.93.2beta.zip
Resolving cnsgenomics.com (cnsgenomics.com)... 203.101.226.237
Connecting to cnsgenomics.com (cnsgenomics.com)|203.101.226.237|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12054383 (11M) [application/zip]
Saving to: ‘gcta_1.93.2beta.zip’

     0K .......... .......... .......... .......... ..........  0%  124K 95s
    50K .......... .......... .......... .......... ..........  0%  248K 71s
   100K .......... .......... .......... .......... ..........  1%  157M 47s
   150K .......... .......... .......... .......... ..........  1%  248K 47s
   200K .......... .......... .......... .......... ..........  2%  126M 37s
   250K .......... .......... .......... .......... ..........  2%  169M 31s
   300K .......... .......... .......... .......... ..........  2%  125M 26s
   350K .......... .......... .......... .......... ..........  3% 1.16M 24s
   400K .......... ..........

In [9]:
!~/bin/gcta_1.93.2beta/gcta64

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version 1.93.2 beta Linux
* (C) 2010-present, Jian Yang, The University of Queensland
* Please report bugs to Jian Yang <jian.yang.qt@gmail.com>
*******************************************************************
Analysis started at 12:49:39 UTC on Tue Feb 02 2021.
Hostname: saturn-6ebc025c-7384-4edf-91c9-1b2092cda50d-m

Error: no analysis has been launched by the option(s)
Please see online documentation at http://cnsgenomics.com/software/gcta


In [10]:
!~/bin/gcta_1.93.2beta/gcta64 --bfile pruned_data --make-grm --out GRM_matrix --autosome --maf 0.05 
!~/bin/gcta_1.93.2beta/gcta64 --grm-cutoff 0.125 --grm GRM_matrix --out GRM_matrix_0125 --make-grm

*******************************************************************
* Genome-wide Complex Trait Analysis (GCTA)
* version 1.93.2 beta Linux
* (C) 2010-present, Jian Yang, The University of Queensland
* Please report bugs to Jian Yang <jian.yang.qt@gmail.com>
*******************************************************************
Analysis started at 12:49:46 UTC on Tue Feb 02 2021.
Hostname: saturn-6ebc025c-7384-4edf-91c9-1b2092cda50d-m

Options: 
 
--bfile pruned_data 
--make-grm 
--out GRM_matrix 
--autosome 
--maf 0.05 

Note: GRM is computed using the SNPs on the autosome.
Reading PLINK FAM file from [pruned_data.fam]...
2688 individuals to be included from FAM file.
2688 individuals to be included. 1503 males, 1185 females, 0 unknown.
Reading PLINK BIM file from [pruned_data.bim]...
904893 SNPs to be included from BIM file(s).
883399 SNPs to be included from valid chromosome number
Threshold to filter variants: MAF > 0.050000.
Computing the genetic relationship matrix (GRM) v2 ...
Subs

In [11]:
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap --keep GRM_matrix_0125.grm.id --make-bed --out AMPPD_after_gender_heterozyg_hapmap_pihat

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_after_gender_heterozyg_hapmap_pihat.log.
Options in effect:
  --bfile AMPPD_after_gender_heterozyg_hapmap
  --keep GRM_matrix_0125.grm.id
  --make-bed
  --out AMPPD_after_gender_heterozyg_hapmap_pihat

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2688 people (1503 males, 1185 females) loaded from .fam.
2688 phenotype values loaded from .fam.
--keep: 2576 people remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
AMPPD_after_gender_heterozyg_hapmap_pihat.hh )

In [12]:
%%bash
cut -f 1,2 AMPPD_after_gender_heterozyg_hapmap_pihat.fam > IDs_before_relatedness_filter.txt
cut -f 1,2 AMPPD_after_gender_heterozyg_hapmap_pihat.fam > IDs_after_relatedness_filter.txt

###Variant QC - Missingness per variant

In [13]:
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap_pihat --make-bed --out AMPPD_after_gender_heterozyg_hapmap_pihat_mind --geno 0.05

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_after_gender_heterozyg_hapmap_pihat_mind.log.
Options in effect:
  --bfile AMPPD_after_gender_heterozyg_hapmap_pihat
  --geno 0.05
  --make-bed
  --out AMPPD_after_gender_heterozyg_hapmap_pihat_mind

60402 MB RAM detected; reserving 30201 MB for main workspace.
28791969 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
AMPPD_after_gender_heterozyg_hapmap_pihat_mind.hh ); many commands treat these


In [14]:
%%bash
grep "(--geno)" AMPPD_after_gender_heterozyg_hapmap_pihat_mind.log > MISSINGNESS_SNPS.txt

Missingness by case control P > 1E-4 # needs case control status

In [15]:
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind --test-missing --out missing_snps 

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to missing_snps.log.
Options in effect:
  --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind
  --out missing_snps
  --test-missing

60402 MB RAM detected; reserving 30201 MB for main workspace.
28224737 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.999631.
28224737 variants and 2576 people pass filters and QC.
Among

In [16]:
%%bash
awk '{if ($5 <= 0.0001) print $2 }' missing_snps.missing > missing_snps_1E4.txt

In [17]:
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind --exclude missing_snps_1E4.txt --make-bed --out AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1.log.
Options in effect:
  --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind
  --exclude missing_snps_1E4.txt
  --make-bed
  --out AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1

60402 MB RAM detected; reserving 30201 MB for main workspace.
28224737 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
--exclude: 28220119 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 don

In [18]:
%%bash
sort -u missing_snps_1E4.txt > VARIANT_TEST_MISSING_SNPS.txt

In [19]:
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1 --test-mishap --out missing_hap 

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to missing_hap.log.
Options in effect:
  --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1
  --out missing_hap
  --test-mishap

60402 MB RAM detected; reserving 30201 MB for main workspace.
28220119 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
commands treat these as missing.
treat these as missing.
Total genotyping rate is 0.999635.
28220119 variants and 2576 people pass filters and QC.

In [20]:
%%bash
awk '{if ($8 <= 0.0001) print $9 }' missing_hap.missing.hap > missing_haps_1E4.txt
sed 's/|/\
/g' missing_haps_1E4.txt > missing_haps_1E4_final.txt

sort -u missing_haps_1E4_final.txt > HAPLOTYPE_TEST_MISSING_SNPS.txt

In [1]:
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1 --exclude missing_haps_1E4_final.txt --make-bed --out AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2.log.
Options in effect:
  --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1
  --exclude missing_haps_1E4_final.txt
  --make-bed
  --out AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2

60402 MB RAM detected; reserving 30201 MB for main workspace.
28220119 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
--exclude: 27938327 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990919293

###Variant QC - HWE

In [2]:
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2 --filter-controls --hwe 1E-4 --write-snplist --out HWE_snps
!~/bin/plink --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2 --extract HWE_snps.snplist --make-bed --out AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing3

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to HWE_snps.log.
Options in effect:
  --bfile AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2
  --filter-controls
  --hwe 1E-4
  --out HWE_snps
  --write-snplist

60402 MB RAM detected; reserving 30201 MB for main workspace.
27938327 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
1615 people removed due to case/control status (--filter-controls).
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 961 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
commands treat these as missing.
treat these as mi

In [3]:
%%bash
mv AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing3.bim AMPPD_FILTERED.bim
mv AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing3.bed AMPPD_FILTERED.bed
mv AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing3.fam AMPPD_FILTERED.fam

#Run PCA on only the AMP-PD to get PCs for the covariate file. 
Download file with long distance high LD-regions to exclude (can alter the PCA if kept)

In [4]:
%%bash
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/exclusion_regions_hg38.txt /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit

Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/exclusion_regions_hg38.txt...
/ [1 files][  466.0 B/  466.0 B]                                                
Operation completed over 1 objects/466.0 B.                                      


In [5]:
# Make sure to use high-quality SNPs and exclude regions of long-range linkage disequilibrium (LD): exclusion_regions_hg38.txt
!~/bin/plink --bfile AMPPD_FILTERED --exclude range exclusion_regions_hg38.txt --autosome --maf 0.01 --geno 0.05 --hwe 1E-6 --make-bed --out EXAMPLE_UNIMPUTED

# Prune out unnecessary SNPs 
!~/bin/plink --bfile EXAMPLE_UNIMPUTED --indep-pairwise 50 5 0.5 --out prune 

# Keep only pruned SNPs
!~/bin/plink --bfile EXAMPLE_UNIMPUTED --extract prune.prune.in --make-bed --out prune 

# Generate PCs 
!~/bin/plink --bfile prune --pca --out EXAMPLE_UNIMPUTED.PCA

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to EXAMPLE_UNIMPUTED.log.
Options in effect:
  --autosome
  --bfile AMPPD_FILTERED
  --exclude range exclusion_regions_hg38.txt
  --geno 0.05
  --hwe 1E-6
  --maf 0.01
  --make-bed
  --out EXAMPLE_UNIMPUTED

60402 MB RAM detected; reserving 30201 MB for main workspace.
26799696 out of 27741772 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
--exclude range: 560772 variants excluded.
--exclude range: 26238924 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899

###Creating covariate file: 
Move over to R and run the code below (do not have the energy to try to do it in python)

Creating covar file

#### Download the necessary packages 
if (!require(tidyverse)) install.packages('tidyverse')
if (!require(data.table)) install.packages('data.table')
if (!require(dplyr)) install.packages('dplyr')
if (!require(plyr)) install.packages('plyr')
if (!require(ggplot2)) install.packages('ggplot2')
if (!require(qqman)) install.packages('qqman')

#### Load the necessary packages 
library(tidyverse)
library(data.table)
library(dplyr)
library(plyr)
library(ggplot2)
library(qqman)

#### Read in the PCA Eigenvalues and Eigenvectors 
eigenvec <- read.delim("EXAMPLE_UNIMPUTED.PCA.eigenvec", sep =" ", header = F, stringsAsFactors = F)
head(eigenvec)

#### Read in the .fam file
fam <- fread("AMPPD_FILTERED.fam", header = F)
head(fam)

#### Read in the covar file (age)
covs <- fread("plink_test_covs.tab", header = T)
head(covs)

#### Rename the columns 
colnames(fam) <- c("FID", "IID", "PAT", "MAT", "SEX", "PHENO")
colnames(eigenvec) <- c("FID", "IID", "PC1", "PC2", "PC3", "PC4", "PC5", "PC6", "PC7", "PC8", "PC9", "PC10", "PC11", "PC12", "PC13", "PC14", "PC15", "PC16", "PC17", "PC18","PC19","PC20")
head(fam)
head(eigenvec)

#Add age to the fam file
fam$AGE = covs$age_at_baseline[match(fam$IID, covs$IID)]
head(fam)

#### Drop the IID column to prevent double columns
eigenvec$FID <- NULL
head (eigenvec)

#### Combine the files
combined <- join(fam, eigenvec, by = c("IID"="IID"))
head(combined)

#### Save the file 
write.table(combined, file = "covariateFile.txt", row.names=FALSE, na="", quote = FALSE, sep="\t")

#### Check that the files seems ok
TEST <- fread("covariateFile.txt", header = T) #should the header be removed??
head(TEST)

### RIC3 analyses

Positions for RIC3:
#hg19 (Only needed for IPDGC data)
chr11:8127597 - 8190602 
8,127,603-8,190,602
#hg38 (What we need for the AMP-PD data)
chr11:8106056-8169055 (Ensemble)

1) Extract the RIC3 region from the filtered data

In [1]:
!~/bin/plink --bfile AMPPD_FILTERED --chr 11 -from-bp 8106056 --to-bp 8169055 --make-bed --out AMPPD_FILTERED.RIC3

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_FILTERED.RIC3.log.
Options in effect:
  --bfile AMPPD_FILTERED
  --chr 11
  --from-bp 8106056
  --make-bed
  --out AMPPD_FILTERED.RIC3
  --to-bp 8169055

60402 MB RAM detected; reserving 30201 MB for main workspace.
773 out of 27741772 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999747.
773 variants and 2576 people pass filters and QC.
Among remaining phenotypes, 1

2) Fisher exact test and logistic regression

Since I added the phenotypes to the plink file, I shouldn't need to set the --pheno? (Doing it anyway)

In [2]:
!~/bin/plink --bfile AMPPD_FILTERED.RIC3 --pheno plink_test_pheno.tab --fisher --out RIC3_WGS_AMP_PD

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to RIC3_WGS_AMP_PD.log.
Options in effect:
  --bfile AMPPD_FILTERED.RIC3
  --fisher
  --out RIC3_WGS_AMP_PD
  --pheno plink_test_pheno.tab

Note: --fisher flag deprecated.  Use '--assoc fisher' or '--model fisher'.
60402 MB RAM detected; reserving 30201 MB for main workspace.
773 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values present after --pheno.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 10111213141516171819202122232425262728293031323334353637383940414243444546474849505152535455565758596061626364656667686970717273747576777879808182838485868788899091929394959697989 done.
Total genotyping rate is 0.999747.
773 variants and 2576 people pass filt

In [3]:
!~/bin/plink --bfile AMPPD_FILTERED.RIC3 --logistic --pheno plink_test_pheno.tab --covar covariateFile.txt --covar-name SEX, AGE, PC1, PC2, PC3, PC4, PC5, PC6, PC7, PC8, PC9, PC10 --out RIC3_WGS_AMP_PD_logreg --ci 0.95

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to RIC3_WGS_AMP_PD_logreg.log.
Options in effect:
  --bfile AMPPD_FILTERED.RIC3
  --ci 0.95
  --covar covariateFile.txt
  --covar-name SEX, AGE, PC1, PC2, PC3, PC4, PC5, PC6, PC7, PC8, PC9, PC10
  --logistic
  --out RIC3_WGS_AMP_PD_logreg
  --pheno plink_test_pheno.tab

60402 MB RAM detected; reserving 30201 MB for main workspace.
773 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values present after --pheno.
Using 1 thread (no multithreaded calculations invoked).
--covar: 12 out of 27 covariates loaded.
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 101112131415161718192021222324252627282930313233343536373839404142434445464748495051525354555657585960616263646566676869707172737475767778798081828384858687888990

Downloading files

In [4]:
!gsutil cp ./RIC3_WGS_AMP_PD.assoc.fisher gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./RIC3_WGS_AMP_PD.log gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./RIC3_WGS_AMP_PD_logreg.assoc.logistic gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./RIC3_WGS_AMP_PD_logreg.log gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf

Copying file://./RIC3_WGS_AMP_PD.assoc.fisher [Content-Type=application/octet-stream]...
/ [1 files][ 63.8 KiB/ 63.8 KiB]                                                
Operation completed over 1 objects/63.8 KiB.                                     
Copying file://./RIC3_WGS_AMP_PD.log [Content-Type=application/octet-stream]...
/ [1 files][  1.0 KiB/  1.0 KiB]                                                
Operation completed over 1 objects/1.0 KiB.                                      
Copying file://./RIC3_WGS_AMP_PD_logreg.assoc.logistic [Content-Type=application/octet-stream]...
/ [1 files][  1.1 MiB/  1.1 MiB]                                                
Operation completed over 1 objects/1.1 MiB.                                      
Copying file://./RIC3_WGS_AMP_PD_logreg.log [Content-Type=application/octet-stream]...
/ [1 files][  1.1 KiB/  1.1 KiB]                                                
Operation completed over 1 objects/1.1 KiB.                                 

### 3) Annotation 

In [5]:
!~/bin/plink --bfile AMPPD_FILTERED.RIC3 --recode 'vcf-fid' --out AMPPD_FILTERED.RIC3

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to AMPPD_FILTERED.RIC3.log.
Options in effect:
  --bfile AMPPD_FILTERED.RIC3
  --out AMPPD_FILTERED.RIC3
  --recode vcf-fid

60402 MB RAM detected; reserving 30201 MB for main workspace.
773 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%39%40%41%42%43%44%45%46%47%48%49%50%51%52%53%54%55%56%57%58%59%60%61%62

Have had some issues with installing rvtests, if the cell below doesn't work, this might

%%bash

if test -e ~/bin/rvtests; then

echo "rvtests is already installed in ~/bin"
else
echo "rvtests is not installed"
cd ~/bin/

wget https://github.com/zhanxw/rvtests/releases/download/v2.1.0/rvtests_linux64.tar.gz

tar -xzvf rvtests_linux64.tar.gz 

fi

In [9]:
%%bash

if test -e ~/bin/rvtests; then

echo "rvtests is already installed in ~/bin"
else
echo "rvtests is not installed"
cd ~/bin/

git clone https://github.com/zhanxw/rvtests

fi

rvtests is already installed in ~/bin


In [ ]:
%%bash
cd ~/bin/rvtests
make

In [11]:
%%bash
cd ~/bin/
ls

annovar
annovar.latest.tar.gz
data_temp
example
executable
gcta_1.93.2beta
gcta_1.93.2beta.zip
LICENSE
plink
plink_linux_x86_64_20190304.zip
prettify
README.md
rvtests
rvtests_linux64.tar.gz
toy.map
toy.ped


In [6]:
%%bash 
rm AMPPD_FILTERED.RIC3.vcf.gz

In [7]:
%%bash
export PATH=$PATH:~/bin/rvtests/third/tabix-0.2.6/
bgzip AMPPD_FILTERED.RIC3.vcf
tabix -f -p vcf AMPPD_FILTERED.RIC3.vcf.gz

Important! You need to add your own download link after registration on the annovar website: https://www.openbioinformatics.org/annovar/annovar_download_form.php

In [30]:
%%bash

if test -e ~/bin/annovar; then

echo "annovar is already installed in ~/bin"
else
echo "annovar is not installed"
cd ~/bin/

wget LINK_TO_ANNOVAR_DOWNLOAD.tar.gz

tar xvfz annovar.latest.tar.gz

fi

annovar is not installed
annovar/
annovar/example/
annovar/example/ex1.avinput
annovar/example/example.simple_region
annovar/example/example.tab_region
annovar/example/ex2.vcf
annovar/example/grantham.matrix
annovar/example/snplist.txt
annovar/example/README
annovar/example/gene_xref.txt
annovar/example/gene_fullxref.txt
annovar/humandb/
annovar/humandb/hg19_refGene.txt
annovar/humandb/hg19_refGeneMrna.fa
annovar/humandb/hg19_refGeneVersion.txt
annovar/humandb/hg19_refGeneWithVer.txt
annovar/humandb/hg19_refGeneWithVerMrna.fa
annovar/humandb/hg19_example_db_generic.txt
annovar/humandb/hg19_example_db_gff3.txt
annovar/humandb/GRCh37_MT_ensGene.txt
annovar/humandb/GRCh37_MT_ensGeneMrna.fa
annovar/humandb/hg19_MT_ensGene.txt
annovar/humandb/hg19_MT_ensGeneMrna.fa
annovar/humandb/genometrax-sample-files-gff/
annovar/humandb/genometrax-sample-files-gff/list
annovar/humandb/genometrax-sample-files-gff/sample_chip_featuretype_hg19.gff
annovar/humandb/genometrax-sample-files-gff/sample_common_

--2021-02-02 15:48:20--  http://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz
Resolving www.openbioinformatics.org (www.openbioinformatics.org)... 67.205.156.247
Connecting to www.openbioinformatics.org (www.openbioinformatics.org)|67.205.156.247|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz [following]
--2021-02-02 15:48:20--  https://www.openbioinformatics.org/annovar/download/0wgxR2rIVP/annovar.latest.tar.gz
Connecting to www.openbioinformatics.org (www.openbioinformatics.org)|67.205.156.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141660305 (135M) [application/x-gzip]
Saving to: ‘annovar.latest.tar.gz’

     0K .......... .......... .......... .......... ..........  0%  919K 2m31s
    50K .......... .......... .......... .......... ..........  0% 1.81M 1m53s
   100K .......... .......... ....

Download needed files:

In [32]:
%%bash
cd ~/bin/annovar/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar refGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb cytoBand humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ensGene humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar exac03 humandb/ 
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar avsnp147 humandb/ 
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar dbnsfp30a humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar gnomad211_genome humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar ljb26_all humandb/
perl annotate_variation.pl -buildver hg38 -downdb -webfrom annovar clinvar_20190305 humandb/

NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGene.txt.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneMrna.fa.gz ... OK
NOTICE: Downloading annotation database http://www.openbioinformatics.org/annovar/download/hg38_refGeneVersion.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether ANNOVAR new version is available ... Done
NOTICE: Downloading annotation database http://hgdownload.cse.ucsc.edu/goldenPath/hg38/database/cytoBand.txt.gz ... OK
NOTICE: Uncompressing downloaded files
NOTICE: Finished downloading annotation files for hg38 build version, with files saved at the 'humandb' directory
NOTICE: Web-based checking to see whether A

In [13]:
%%bash
cd ~/bin/annovar/humandb/
ls

annovar_downdb.log
genometrax-sample-files-gff
GRCh37_MT_ensGeneMrna.fa
GRCh37_MT_ensGene.txt
hg19_example_db_generic.txt
hg19_example_db_gff3.txt
hg19_MT_ensGeneMrna.fa
hg19_MT_ensGene.txt
hg19_refGeneMrna.fa
hg19_refGene.txt
hg19_refGeneVersion.txt
hg19_refGeneWithVerMrna.fa
hg19_refGeneWithVer.txt
hg38_avsnp147.txt
hg38_avsnp147.txt.idx
hg38_clinvar_20190305.txt
hg38_clinvar_20190305.txt.idx
hg38_cytoBand.txt
hg38_dbnsfp30a.txt
hg38_dbnsfp30a.txt.idx
hg38_ensGeneMrna.fa
hg38_ensGene.txt
hg38_exac03.txt
hg38_exac03.txt.idx
hg38_gnomad211_genome.txt
hg38_gnomad211_genome.txt.idx
hg38_ljb26_all.txt
hg38_ljb26_all.txt.idx
hg38_refGeneMrna.fa
hg38_refGene.txt
hg38_refGeneVersion.txt


There is something that isn't correct in the input vcf I think since I get >1000 columns with "Otherinfo" when using the vcf file. I decided to change to format to a "annovar formated file" with the code below and everything appears to look fine

#The following code generates a file with >1000 "otherinfo" columns, there is something wrong with the vcf (after bgzip and tabix), hence I converted it to the "annovar format" instead

%%bash
perl ~/bin/annovar/table_annovar.pl AMPPD_FILTERED.RIC3.vcf.gz ~/bin/annovar/humandb/ -buildver hg38 \
--thread 16 \
-out RIC3_WGS.annovar \
-remove -protocol refGene,ljb26_all,gnomad211_genome,clinvar_20190305 \
-operation g,f,f,f \
-nastring . \
-vcfinput

In [12]:
%%bash

perl ~/bin/annovar/convert2annovar.pl -format vcf4 -allsample -withfreq AMPPD_FILTERED.RIC3.vcf.gz > AMPPD_FILTERED.RIC3_Annovar

NOTICE: Finished reading 780 lines from VCF file
NOTICE: A total of 773 locus in VCF file passed QC threshold, representing 719 SNPs (477 transitions and 242 transversions) and 50 indels/substitutions
NOTICE: Finished writing allele frequencies based on 1852144 SNP genotypes (1228752 transitions and 623392 transversions) and 128800 indels/substitutions for 2576 samples


In [13]:
%%bash

perl ~/bin/annovar/table_annovar.pl AMPPD_FILTERED.RIC3_Annovar ~/bin/annovar/humandb/ -buildver hg38 \
--thread 16 \
-out RIC3_WGS.annovar \
-remove -protocol refGene,ljb26_all,gnomad211_genome,clinvar_20190305 \
-operation g,f,f,f \
-nastring .

NOTICE: the --polish argument is set ON automatically (use --nopolish to change this behavior)
-----------------------------------------------------------------
NOTICE: Processing operation=g protocol=refGene

NOTICE: Running with system command <annotate_variation.pl -geneanno -buildver hg38 -dbtype refGene -outfile RIC3_WGS.annovar.refGene -exonsort -nofirstcodondel AMPPD_FILTERED.RIC3_Annovar /home/jupyter-user/bin/annovar/humandb/ -thread 16>
NOTICE: Output files are written to RIC3_WGS.annovar.refGene.variant_function, RIC3_WGS.annovar.refGene.exonic_variant_function
NOTICE: the queryfile AMPPD_FILTERED.RIC3_Annovar contains 773 lines
NOTICE: threading is disabled for gene-based annotation on file with less than 1000000 input lines
NOTICE: Reading gene annotation from /home/jupyter-user/bin/annovar/humandb/hg38_refGene.txt ... Done with 82500 transcripts (including 20366 without coding sequence annotation) for 28265 unique genes
NOTICE: Processing next batch with 773 unique varian

In [14]:
%%bash
head -1 RIC3_WGS.annovar.hg38_multianno.txt > header.txt
colct="$(wc -w header.txt| cut -f1 -d' ')"
cut -f1-$colct RIC3_WGS.annovar.hg38_multianno.txt > RIC3.WGS.trimmed.annotation_KB.txt

In [15]:
#download the annotation file
!gsutil cp ./RIC3.WGS.trimmed.annotation_KB.txt gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf

Copying file://./RIC3.WGS.trimmed.annotation_KB.txt [Content-Type=text/plain]...
/ [1 files][166.5 KiB/166.5 KiB]                                                
Operation completed over 1 objects/166.5 KiB.                                    


#### 4) Burden analysis
##### Generating list of coding variants

In [16]:
%%bash

awk '$6=="exonic" {print}' RIC3.WGS.trimmed.annotation_KB.txt > RIC3.WGS.trimmed.annotation_KB.coding.variants.txt
awk '{print $1" "$2" "$2" "$7}' RIC3.WGS.trimmed.annotation_KB.coding.variants.txt > RIC3.WGS.trimmed.annotation_KB.SNPs.txt


In [17]:
!~/bin/plink --bfile AMPPD_FILTERED.RIC3 --extract range RIC3.WGS.trimmed.annotation_KB.coding.variants.txt --recode 'vcf-fid'  --out RIC3_CODING_KB

PLINK v1.90b6.9 64-bit (4 Mar 2019)            www.cog-genomics.org/plink/1.9/
(C) 2005-2019 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to RIC3_CODING_KB.log.
Options in effect:
  --bfile AMPPD_FILTERED.RIC3
  --extract range RIC3.WGS.trimmed.annotation_KB.coding.variants.txt
  --out RIC3_CODING_KB
  --recode vcf-fid

60402 MB RAM detected; reserving 30201 MB for main workspace.
773 variants loaded from .bim file.
2576 people (1455 males, 1121 females) loaded from .fam.
2576 phenotype values loaded from .fam.
--extract range: 754 variants excluded.
--extract range: 19 variants remaining.
Using 1 thread (no multithreaded calculations invoked).
Before main variant filters, 2576 founders and 0 nonfounders present.
Calculating allele frequencies... 0%1%2%3%4%5%6%7%8%9%10%11%12%13%14%15%16%17%18%19%20%21%22%23%24%25%26%27%28%29%30%31%32%33%34%35%36%37%38%3

MAF < 0.03
#### ALL VARIANTS 

##Upload needed file:

In [18]:
%%bash
gsutil cp gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/refFlat.txt /home/jupyter-user/notebooks/ipdgctraineeproject-ric3/edit

Copying gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf/refFlat.txt...
/ [1 files][ 21.2 MiB/ 21.2 MiB]                                                
Operation completed over 1 objects/21.2 MiB.                                     


In [18]:
%%bash
export PATH=$PATH:~/bin/executable
rvtest --noweb --inVcf AMPPD_FILTERED.RIC3.vcf.gz --pheno covariateFile.txt --pheno-name PHENO --covar covariateFile.txt --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --burden cmc,zeggini,mb,fp,cmcWald --geneFile refFlat.txt --freqUpper 0.03 --out AMP_PD_BURDEN.RIC3.WGS.maf003_KB

Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please submit at: https://github.com/zhanxw/rvtests/issues

RVTESTS finished successfully


The following parameters are available.  Ones with "[]" are in effect:

Available Options
      Basic Input/Output: --inVcf [AMPPD_FILTERED.RIC3.vcf.gz], --inBgen []
                          --inBgenSample [], --inKgg []
                          --out [AMP_PD_BURDEN.RIC3.WGS.maf003_KB], --outputRaw
       Specify Covariate: --covar [covariateFile.txt], --covar-name [SEX,AGE,PC1
                         PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10], --sex
       Specify Phenotype: --pheno [covariateFile.txt], --inverseNormal
                          --useResidualAsPhenotype, --mpheno []
                          --pheno-name [PHENO], --qtl, --multiplePheno []
        Specify Genotype: --dosage [], --multipleAllele
    Chromosome X Options: --xLabel [], --xParRegion []
           People Filter: --peopleIncludeID [], --peopleIncludeFile []
                          --peopleExcludeID [], --peopleExcludeFile []
             Site Filter: --rangeList [], --rangeFile [], --siteFile []
             

#### CODING VARIANTS 

In [20]:
%%bash
export PATH=$PATH:~/bin/rvtests/third/tabix-0.2.6/
bgzip RIC3_CODING_KB.vcf
tabix -f -p vcf RIC3_CODING_KB.vcf.gz

In [21]:
%%bash
export PATH=$PATH:~/bin/executable
rvtest --noweb --inVcf RIC3_CODING_KB.vcf.gz --pheno covariateFile.txt --pheno-name PHENO --covar covariateFile.txt --covar-name SEX,AGE,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10 --kernel skat,skato --burden cmc,zeggini,mb,fp,cmcWald --geneFile refFlat.txt --freqUpper 0.03 --out AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING

Thank you for using rvtests (version: 20190205, git: c86e589efef15382603300dc7f4c3394c82d69b8)
  For documentations, refer to http://zhanxw.github.io/rvtests/
  For questions and comments, plase send to Xiaowei Zhan <zhanxw@umich.edu>
  For bugs and feature requests, please submit at: https://github.com/zhanxw/rvtests/issues

RVTESTS finished successfully


The following parameters are available.  Ones with "[]" are in effect:

Available Options
      Basic Input/Output: --inVcf [RIC3_CODING_KB.vcf.gz], --inBgen []
                          --inBgenSample [], --inKgg []
                          --out [AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING]
                          --outputRaw
       Specify Covariate: --covar [covariateFile.txt], --covar-name [SEX,AGE,PC1
                         PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10], --sex
       Specify Phenotype: --pheno [covariateFile.txt], --inverseNormal
                          --useResidualAsPhenotype, --mpheno []
                          --pheno-name [PHENO], --qtl, --multiplePheno []
        Specify Genotype: --dosage [], --multipleAllele
    Chromosome X Options: --xLabel [], --xParRegion []
           People Filter: --peopleIncludeID [], --peopleIncludeFile []
                          --peopleExcludeID [], --peopleExcludeFile []
             Site Filter: --rangeList [], --rangeFile [], 

Download files:

In [22]:
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.CMC.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.CMCWald.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.CMC.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.CMCWald.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.Fp.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.log gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.MadsonBrowning.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.Skat.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.SkatO.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.Zeggini.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.Fp.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.log gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.MadsonBrowning.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.Skat.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.SkatO.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf
!gsutil cp ./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.Zeggini.assoc gs://fc-secure-d273a1b3-b576-476f-995b-1b251495c9bf

Copying file://./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.CMC.assoc [Content-Type=application/octet-stream]...
/ [1 files][  321.0 B/  321.0 B]                                                
Operation completed over 1 objects/321.0 B.                                      
Copying file://./AMP_PD_BURDEN.RIC3.WGS.maf003_KB.CMCWald.assoc [Content-Type=application/octet-stream]...
/ [1 files][  3.6 KiB/  3.6 KiB]                                                
Operation completed over 1 objects/3.6 KiB.                                      
Copying file://./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.CMC.assoc [Content-Type=application/octet-stream]...
/ [1 files][  318.0 B/  318.0 B]                                                
Operation completed over 1 objects/318.0 B.                                      
Copying file://./AMP_PD_BURDEN.RIC3.WGS.maf003_KB_CODING.CMCWald.assoc [Content-Type=application/octet-stream]...
/ [1 files][  3.6 KiB/  3.6 KiB]                                             

Remove non-needed files

In [49]:
%%bash
ls

=1.13.0
afri_add.txt
AFRICA_confirmation_plot.pdf
afrio.txt
afri.txt
after_gender3.bed
after_gender3.bim
after_gender3.fam
after_gender3.hh
after_gender3.log
after_gender4.bed
after_gender4.bim
after_gender4.fam
after_gender4.hh
after_gender4.log
all_outliers.txt
AMPPD_after_gender.bed
AMPPD_after_gender.bim
AMPPD_after_gender.fam
AMPPD_after_gender_heterozyg_hapmap.bed
AMPPD_after_gender_heterozyg_hapmap.bim
AMPPD_after_gender_heterozyg_hapmap.fam
AMPPD_after_gender_heterozyg_hapmap.hh
AMPPD_after_gender_heterozyg_hapmap.log
AMPPD_after_gender_heterozyg_hapmap_pihat.bed
AMPPD_after_gender_heterozyg_hapmap_pihat.bim
AMPPD_after_gender_heterozyg_hapmap_pihat.fam
AMPPD_after_gender_heterozyg_hapmap_pihat.hh
AMPPD_after_gender_heterozyg_hapmap_pihat.log
AMPPD_after_gender_heterozyg_hapmap_pihat_mind.bed
AMPPD_after_gender_heterozyg_hapmap_pihat_mind.bim
AMPPD_after_gender_heterozyg_hapmap_pihat_mind.fam
AMPPD_after_gender_heterozyg_hapmap_pihat_mind.hh
AMPPD_after_gender_heterozyg_hapmap_

In [50]:
%%bash
rm after_gender3.bed
rm after_gender3.bim
rm after_gender3.fam
rm after_gender3.hh
rm after_gender3.log
rm after_gender4.bed
rm after_gender4.bim
rm after_gender4.fam
rm after_gender4.hh
rm after_gender4.log
rm all_outliers.txt
rm AMPPD_after_gender.bed
rm AMPPD_after_gender.bim
rm AMPPD_after_gender.fam
rm AMPPD_after_gender_heterozyg_hapmap.bed
rm AMPPD_after_gender_heterozyg_hapmap.bim
rm AMPPD_after_gender_heterozyg_hapmap.fam
rm AMPPD_after_gender_heterozyg_hapmap.hh
rm AMPPD_after_gender_heterozyg_hapmap.log
rm AMPPD_after_gender_heterozyg_hapmap_pihat.bed
rm AMPPD_after_gender_heterozyg_hapmap_pihat.bim
rm AMPPD_after_gender_heterozyg_hapmap_pihat.fam
rm AMPPD_after_gender_heterozyg_hapmap_pihat.hh
rm AMPPD_after_gender_heterozyg_hapmap_pihat.log
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind.bed
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind.bim
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind.fam
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind.hh
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind.log
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1.bed
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1.bim
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1.fam
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1.hh
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing1.log
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2.bed
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2.bim
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2.fam
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2.hh
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing2.log
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing3.hh
rm AMPPD_after_gender_heterozyg_hapmap_pihat_mind_missing3.log
rm AMPPD_after_gender.hh
rm AMPPD_after_gender.log
rm AMPPD_after_heterozyg.bed
rm AMPPD_after_heterozyg.bim
rm AMPPD_after_heterozyg.fam
rm AMPPD_after_heterozyg.hh
rm AMPPD_after_heterozyg.log
rm AMPPD_call_rate.bed
rm AMPPD_call_rate.bim
rm AMPPD_call_rate.fam
rm AMPPD_call_rate.hh
rm AMPPD_call_rate.log
rm AMPPD_updated_pheno.bed
rm AMPPD_updated_pheno.bim
rm AMPPD_updated_pheno.fam
rm AMPPD_updated_pheno.hh
rm AMPPD_updated_pheno.log
rm call_rates.hh
rm call_rates.imiss
rm call_rates.lmiss
rm call_rates.log
rm EXAMPLE_UNIMPUTED.bed
rm EXAMPLE_UNIMPUTED.bim
rm EXAMPLE_UNIMPUTED.fam
rm EXAMPLE_UNIMPUTED.log
rm gender_check1.hh
rm gender_check1.log
rm gender_check1.sexcheck
rm gender_check2.hh
rm gender_check2.log
rm gender_check2.sexcheck
rm GENDER_FAILURES.txt
rm GRM_matrix_0125.grm.bin
rm GRM_matrix_0125.grm.id
rm GRM_matrix_0125.grm.N.bin
rm GRM_matrix_0125.log
rm GRM_matrix.grm.bin
rm GRM_matrix.grm.id
rm GRM_matrix.grm.N.bin
rm GRM_matrix.log
rm HAPLOTYPE_TEST_MISSING_SNPS.txt
rm hapmap3_bin_snplis.bed
rm hapmap3_bin_snplis.bim
rm hapmap3_bin_snplis.fam
rm hapmap3_bin_snplis.hh
rm hapmap3_bin_snplis.log
rm hapmap3_bin_snplis-merge.missnp
rm hapmap_outliers33.txt
rm header.txt
rm HETEROZYGOSITY_DATA.txt
rm HETEROZYGOSITY_OUTLIERS.txt
rm HWE_snps.hh
rm HWE_snps.log
rm HWE_snps.snplist
rm IDs_after_relatedness_filter.txt
rm IDs_before_relatedness_filter.txt
rm missing_hap.hh
rm missing_hap.log
rm missing_hap.missing.hap
rm missing_haps_1E4_final.txt
rm missing_haps_1E4.txt
rm MISSINGNESS_SNPS.txt
rm missing_snps_1E4.txt
rm missing_snps.hh
rm missing_snps.log
rm missing_snps.missing
rm mixed_race_confirmation_plot.pdf
rm new_samples2.txt
rm new_samples_add.txt
rm new_samples.txt
rm outliers1.txt
rm outliers2.txt
rm pca.bed
rm pca.bim
rm pca.fam
rm pca.log
rm problems1.txt
rm problems2.txt
rm prune.bed
rm prune.bim
rm pruned_data.bed
rm pruned_data.bim
rm pruned_data.fam
rm pruned_data.hh
rm pruned_data.log
rm prunedHet.het
rm prunedHet.hh
rm prunedHet.log
rm prune.fam
rm prune.log
rm prune.prune.in
rm prune.prune.out
rm pruning.hh
rm pruning.log
rm pruning.prune.in
rm pruning.prune.out
rm samples_to_remove.txt
rm VARIANT_TEST_MISSING_SNPS.txt